# 1. 单局 PK 可视化区域

In [ ]:
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy

In [ ]:
from gomoku.player import IneffectiveZeroMCTSPlayer


player1 = ZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=300, device='cpu', eager=False)
player2 = ZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=300, device='cpu', eager=False)

game = GomokuEnvSimple()

_ = play_one_game(player1, player2, game, render=True)

# 2. ARENA 

In [ ]:
from gomoku.player import arena_parallel
from gomoku.player import ZeroMCTSPlayer, WrongZeroMCTSPlayer, IneffectiveZeroMCTSPlayer, play_one_game
from gomoku.gomoku_env import GomokuEnvSimple
from gomoku.policy import ZeroPolicy

In [ ]:
r = arena_parallel(
    player1_main=ZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=50, device='cpu', eager=False),
    player2_main=WrongZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=50, device='cpu', eager=False),
    games=100,
)

In [ ]:
r = arena_parallel(
    player1_main=ZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=100, device='cpu', eager=False),
    player2_main=IneffectiveZeroMCTSPlayer(ZeroPolicy(board_size=9), itermax=100, device='cpu', eager=False),
    games=50,
)
